## Preambule

In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import xarray as xr
import plotly.graph_objects as go

## Read data

In [12]:
xr_total = xr.open_dataset(Path("X:/user/dekkerm/Projects/ECEMF/WP5/") / "Data" / "EffortSharing" / "1990" / "xr_total.nc")
xr_budgets_scenario = xr.open_dataset(Path("X:/user/dekkerm/Projects/ECEMF/WP5/") / "Data" / "EffortSharing" / "1990" / "xr_budgets_scenario.nc")
xr_budgets_static = xr.open_dataset(Path("X:/user/dekkerm/Projects/ECEMF/WP5/") / "Data" / "EffortSharing" / "1990" / "xr_budgets_static.nc")

## Check: missing data

In [4]:
df_missing = {}
df_missing['ISO3'] = xr_total.ISO
df_missing["Population_past"] = np.isnan(np.array(xr_total.Population.sel(Time=2000)))
df_missing["Population_future"] = np.isnan(np.array(xr_total.Population.sel(Time=2050)))
df_missing["GDP_past"] = np.isnan(np.array(xr_total.GDP.sel(Time=2000)))
df_missing["GDP_future"] = np.isnan(np.array(xr_total.GDP.sel(Time=2050)))
df_missing["Emissions_past"] = np.isnan(np.array(xr_total.GHG_p.sel(Time=2000)))
df_missing["HDI"] = np.isnan(np.array(xr_total.HDI))
df_missing["NDC"] = np.isnan(np.array(xr_total.NDC_h.sel(Time=2030)))

In [5]:
pd.DataFrame(df_missing).to_csv(Path("X:/user/dekkerm/Projects/ECEMF/WP5/") / "Data" / "EffortSharing" / "MissingData.csv")

In [6]:
for k in list(df_missing.keys())[1:]:
    print(k+':', np.round(np.sum(df_missing[k])/len(df_missing[k])*100,2), 'percent is missing')
print()
pop = np.array(xr_total.Population.sel(Time=2020))
for k in list(df_missing.keys())[1:]:
    popw = np.array(xr_total.Population.sel(Time=2020, ISO='WORLD'))
    print(k+':', np.round(np.nansum(df_missing[k]*1*pop)/popw*100,5), 'percent is missing')

Population_past: 5.34 percent is missing
Population_future: 5.34 percent is missing
GDP_past: 16.79 percent is missing
GDP_future: 16.79 percent is missing
Emissions_past: 16.79 percent is missing
HDI: 22.14 percent is missing
NDC: 20.99 percent is missing

Population_past: 0.0 percent is missing
Population_future: 0.0 percent is missing
GDP_past: 0.35062 percent is missing
GDP_future: 0.35062 percent is missing
Emissions_past: 0.16539 percent is missing
HDI: 0.35499 percent is missing
NDC: 0.59889 percent is missing


## Check: Fyson 2 behavior per country

## Check monotomy

In [11]:
xr_total.sel(Category='C1', ISO='USA')

<xarray.Dataset>
Dimensions:     (Time: 111)
Coordinates:
    ISO         <U3 'USA'
  * Time        (Time) int32 1990 1991 1992 1993 1994 ... 2097 2098 2099 2100
    Category    <U2 'C1'
Data variables: (12/14)
    Population  (Time) float64 2.464e+08 2.498e+08 ... 3.939e+08 3.94e+08
    GDP         (Time) float64 nan nan nan nan ... 4.273e+13 4.312e+13 4.351e+13
    GHG_e       (Time) float64 6.234 6.193 6.292 6.418 6.537 ... nan nan nan nan
    GHG_p       (Time) float64 6.109 6.1 6.203 6.318 6.42 ... nan nan nan nan
    GHG_p_incl  (Time) float64 5.693 5.683 5.787 5.901 6.004 ... nan nan nan nan
    GHG_base    (Time) float64 nan nan nan nan nan ... 5.184 5.165 5.146 5.127
    ...          ...
    HDI         float64 0.921
    HDIsh       float64 nan
    NegGHG      (Time) float64 nan nan nan nan nan ... 19.74 18.9 18.07 17.24
    GHG_f       (Time) float64 nan nan nan nan nan ... 4.311 4.06 3.81 3.56
    GHG_f_incl  (Time) float64 nan nan nan nan ... 0.707 0.4375 0.1681 -0.1014
    GHG_f_co2   (Time) float64 nan nan nan nan ... -6.092 -6.268 -6.444 -6.62

In [22]:
float(xr_budgets_static.sel(Region="USA", Ccat="C1", Variable="A1_gdp_net").Value)

-102.50740455974064

In [44]:
def check_monotomy(reg, var):
    y0 = -1e8
    buds = []
    for c_i, c in enumerate(['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8']):
        y1 = float(xr_budgets_static.sel(Region=reg, Ccat=c, Variable=var).Value)
        buds.append(y1)
        if y1 > y0:
            y0 = float(np.copy(y1))
            continue
        else:
            return 0
    return 1

In [ ]:
def check_monotomy_dynamic(reg, var):
    y0 = -1e8
    buds = []
    for c_i, c in enumerate(['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8']):
        y1 = float(xr_budgets_static.sel(Region=reg, Ccat=c, Variable=var).Value)
        buds.append(y1)
        if y1 > y0:
            y0 = float(np.copy(y1))
            continue
        else:
            return 0
    return 1

In [52]:
xr_budgets_scenario.sel(Variable='A1_gdp_net', Region='USA')

<xarray.Dataset>
Dimensions:   (Time: 81, Ccat: 12)
Coordinates:
    Region    <U3 'USA'
  * Time      (Time) int32 2020 2021 2022 2023 2024 ... 2096 2097 2098 2099 2100
    Variable  <U10 'A1_gdp_net'
  * Ccat      (Ccat) object 'C1' 'C1+C2' 'C2' 'C3' ... 'C6' 'C7' 'C7+C8' 'C8'
Data variables:
    Value     (Ccat, Time) float64 ...

In [48]:
check_monotomy('CHN', 'A1_gdp_net')

(1,
 [130.69146637397736,
  152.55517471552787,
  190.24950714527085,
  256.99529184740226,
  323.7410765495336,
  453.79702708329506,
  621.8152381779416,
  828.1553411384826])

In [33]:
fig = go.Figure()
for c_i, c in enumerate(['C1', 'C2', 'C3']):
    bars = go.Bar(name=c,
                  x=['USA', 'EU', "CHN"],
                  y=xr_budgets_static.sel(Region=['USA', 'EU', "CHN"], Ccat=c, Variable='A1_gdp_net').Value)
    fig.add_trace(bars)
fig.update_layout(barmode='group')
fig.show()